# 在Pytorch中实现神经网络

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    """定义神经网络结构，输入数据 1*32*32 """
    def __init__(self):
        super(Net, self).__init__()
        # 第1层 卷积层
        self.conv1 = nn.Conv2d(1,6,3) # 输入频道1，输出频道6，卷积3*3
        # 第2层 卷积层
        self.conv2 = nn.Conv2d(6, 16, 3) # 输入频道6， 输出频道16， 卷积3*3
        # 第3层 全连接层
        self.fc1 = nn.Linear(16*28*28, 512) # 输入维度 16*28*28 输出维度 512
        # 第4层 全连接层
        self.fc2 = nn.Linear(512, 64) #
        # 第5层 全连接层
        self.fc3 = nn.Linear(64,2)
    def forward(self,x): # 定义数据流向
        x = self.conv1(x)
        x = F.relu(x) # 使用激活函数固定数据到一个范围

        x = self.conv2(x)
        x = F.relu(x)

        x = x.view(-1, 16*28*28) # 展开
        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=12544, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)


## 运行神经网络

In [4]:
# 1 定义一个输入数据
input_data = torch.randn(1,1,32,32) # 生成随机数据
print(input_data)
print(input_data.size())

tensor([[[[-1.3800,  0.6739, -0.7494,  ...,  1.8601,  0.3926, -2.5112],
          [ 0.0231,  1.0766, -0.2427,  ..., -1.6085,  0.1639,  0.2657],
          [ 0.2162,  0.0960, -0.3115,  ...,  1.3169, -0.6014,  0.7462],
          ...,
          [-1.7108, -0.4919,  0.0113,  ..., -1.9979, -0.9767, -0.7814],
          [ 1.0260, -1.9570, -0.7377,  ..., -0.0562,  1.2215, -0.1721],
          [ 0.6716, -0.2941,  0.0355,  ..., -0.0763, -0.0448, -0.0812]]]])
torch.Size([1, 1, 32, 32])


In [5]:
# 2 运行神经网络
out = net(input_data)
print(out)
print(out.size())

tensor([[ 0.0353, -0.0335]], grad_fn=<AddmmBackward>)
torch.Size([1, 2])
tensor([[0.2248, 0.4287]])
tensor(0.3259, grad_fn=<L1LossBackward>)


In [6]:
# 3 计算损失
target = torch.randn(2) # 随机生成 真实值
target = target.view(1,-1)
print(target)

tensor([[ 0.1530, -0.7423]])


In [7]:
criterion = nn.L1Loss() # 定义损失函数 绝对值函数
loss = criterion(out, target) # 计算损失
print(loss)

tensor(0.4133, grad_fn=<L1LossBackward>)


In [8]:
# 4 反向传递
net.zero_grad() # 清零梯度
loss.backward()  # 自动梯度计算,反向传递

In [9]:
# 5 权重更新
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01) # 使用SGD优化器去更新所有的权重
optimizer.step()

tensor([[ 0.0808, -0.0881]], grad_fn=<AddmmBackward>)
torch.Size([1, 2])
tensor(0.3632, grad_fn=<L1LossBackward>)


In [10]:
# 6 重新计算
out = net(input_data)
print(out)
print(out.size())

loss = criterion(out, target) # 重新计算损失
print(loss)

tensor([[ 0.0808, -0.0881]], grad_fn=<AddmmBackward>)
torch.Size([1, 2])
tensor(0.3632, grad_fn=<L1LossBackward>)
